In [ ]:
from langchain.llms import OpenAI

import os
llm = OpenAI(openai_api_key=os.environ["OPEN_API_KEY"],temperature=0.6)
text = "what is the capital of India"
print(llm.invoke(text))
kavya = os.environ.get('HUGGINGFACEHUB_API_TOKEN')
print(kavya)
from langchain import HuggingFaceHub
llm_hugging_face = HuggingFaceHub(repo_id="google/flan-t5-large", model_kwargs={"temperature":0.5, "max_length":64 })
output = llm_hugging_face.invoke("can you write a poem about AI")
output
llm.invoke("can you write a poem about AI")

# Prompt Templates & LLM Chains
from langchain.prompts import PromptTemplate
prompt_template = PromptTemplate(input_variables=['Country'], template = "What is the capital of this {country}")
prompt_template.format(country = "India")
from langchain.chains import LLMChain
chain = LLMChain(llm = llm, prompt=prompt_template)
chain.invoke("India")
# combining Multiple chains using simple sequential chain
capital_template = PromptTemplate(input_variables=['country'], template="Give me the capital of {country}")
capital_chain = LLMChain(llm = llm, prompt=capital_template)
famous_template = PromptTemplate(input_variables=['capital'], template="Suggest me some places to visit in {capital}")
famous_chain = LLMChain(llm = llm, prompt=famous_template)
from langchain.chains import SimpleSequentialChain
chain = SimpleSequentialChain(chains = [capital_chain, famous_chain])
chain.invoke("India")
# Sequence Chain
capital_template = PromptTemplate(input_variables=['country'], template="Give me the capital of {country}")
capital_chain = LLMChain(llm = llm, prompt=capital_template, output_key="capital")

neighbour_template = PromptTemplate(input_variables = ['country'], template = "Give me the neighbouring states to {capital}")
neighbour_chain = LLMChain(llm = llm, prompt=neighbour_template, output_key="Neighbours")
from langchain.chains import SequentialChain
chain = SequentialChain(chains=[capital_chain, neighbour_chain], input_variables=['country'], output_variables=['capital', 'Neighbours'])
# chain({'country':"India"})
chain.invoke('India')
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage
chatllm = ChatOpenAI(openai_api_key=os.environ["OPEN_API_KEY"], temperature = 0.5, model = 'gpt-3.5-turbo')

chatllm
chatllm([
    SystemMessage(content="you are the dietician"),
    HumanMessage(content="please provide an Indian diet plan for 7 days to reduce weight. Use only vegetarian foods. give me quantiti too")
])
# Prompt Template + LLM + Output Parsers
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate
from langchain.schema import BaseOutputParser
class commaSeperatedOutput(BaseOutputParser):
    def parse(self, text: str):
        return text.strip().split(",")
template = "Your are a helpful assistant. When user gives any input, you should generate 5 synonym words with comma seperated"
human_template = "{text}"
chatprompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", human_template) 
])
chain = chatprompt|chatllm|commaSeperatedOutput()
chain.invoke({"text":"Intelligent"})
